In [11]:
from dotenv import load_dotenv
import sys

sys.path.append("../../common")
load_dotenv()

True

In [6]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response
from multimodal import MultiModal
from tools_news import GoogleNews

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="17.LangGraph")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 17.LangGraph]


### LangGraph 에서 자주 사용하는 문법들

#### TypedDict

`dict`와 `TypedDict` 의 차이점과 LangGragh에서 `TypedDict`를 사용하는 이유에 대한 설명입니다.

1. `dict`와 `TypedDict` 의 차이점:

    - 타입 검사:
        - `dict` : 런타임에서 타입 검사를 하지 않습니다.
        - `TypedDict` : 런타임에서 타입 검사를 합니다. 런타임에서 검사를 한다는 것은 IDE에서 타입 검사를 해주는 것을 의미합니다.
        
    - 키와 값의 타입
        - `dict` : 키와 값의 타입을 일반적으로 지정합니다. (ex. `dict[str, int]`)
        - `TypedDict` : 각 키에 대해 구체적인 타입을 지정할 수 있습니다. (ex. `TypedDict('MyDict', {'name': str, 'age': int})`)

    - 유연성
        - `dict` : 런타임에 키를 추가하거나 제거할 수 있습니다.
        - `TypedDict` : 정의된 구조를 따라야 합니다. 추가적인 키는 타입오류를 발생시킵니다.

2. LangGraph에서 `TypedDict`를 사용하는 이유:
    - 타입 안정성 : 엄격한 타입 검사를 제공하여 잠재적인 버그를 미리 방지할 수 있습니다.
    - 코드 가독성 : 딕셔너리의 구조를 명확하게 정의할 수 있어 코드의 가독성이 향상됩니다.
    - IDE지원, 디버깅 용이성 : IDE에서 자동 완성 및 타입 힌트를 더 정확하게 제공받을 수 있고, 타입 오류를 미리 발견하여 디버깅을 용이하게 합니다.
    - 협업 향상 : 타입 정의를 공유하여 팀원들 간의 협업을 용이하게 합니다. (문서화)

##### 사용 예시

In [9]:
# Dict와 TypedDict 차이점 예시
from typing import Dict, TypedDict

# Dict 예시
a = dict()
a["name"] = "John"
a


# TypedDict 예시
class Person(TypedDict):
    name: str
    age: int
    job: str


herry = Person(name="Sui", age=20, job="Student")
john: Person = {"name": "John", "age": 30, "job": "Developer"}

{'name': 'John', 'age': 30, 'job': 'Developer'}

#### Annotated

타입 힌트를 추가하는 방법입니다. 파이썬코드에서 코드에 대한 주석을 붙이듯이 타입에 대한 힌트를 제공하는 문법입니다.  

##### Annotated 주요 기능
1. 추가정보 제공: 타입 힌트에 메타데이터를 추가하여 더 상세한 정보를 제공합니다.
2. 문서화: 코드 자체에 추가 설명을 포함시켜 문서화 효과를 얻습니다.
3. 유효성 검사: Pydantic과 함께 사용하여 데이터 유효성 검사를 수행합니다.
4. 프레임워크 지원: LangGragh에서는 Annotated를 사용하여 타입 힌트를 제공합니다.

##### 사용 예시

기본 사용하는 예시입니다.

In [10]:
from typing import Annotated

name: Annotated[str, "사용자 이름"]
age: Annotated[int, "사용자 나이(0-150)"]

Pydantic과 함께 사용하는 예시입니다.

In [3]:
from typing import Annotated, List
from pydantic import BaseModel, Field, ValidationError


class Employee(BaseModel):
    id: Annotated[int, Field(..., description="직원 ID")]
    name: Annotated[str, Field(..., description="직원 이름")]
    age: Annotated[int, Field(..., description="직원 생년월일")]
    salary: Annotated[
        int, Field(gt=0, lt=100000, description="직원 연봉(단위: 만원, 최대 10억)")
    ]
    skills: Annotated[
        List[str], Field(min_item=1, max_items=10, description="보유 기술(1-10개")
    ]


# 유요한 데이터로 객체 생성
try:
    valid_employee = Employee(
        id=1, name="John", age=30, salary=50000, skills=["Python", "Java"]
    )
except ValidationError as e:
    print("유효성 검사 오류: ", e)

# 유효하지 않은 데이터로 객체 생성
try:
    invalid_employee = Employee(
        id=1, name="John", age=30, salary=200000, skills=["Python", "Java"]
    )
except ValidationError as e:
    print("유효성 검사 오류: ")
    for error in e.errors():
        print(f"필드: {error['loc'][0]}, 오류: {error['msg']}")

유효성 검사 오류: 
필드: salary, 오류: Input should be less than 100000


#### LangGraph에서의 TypedDict, Annotated 사용(add_messages)

`add_messages` 함수는 메시지를 추가하는 함수입니다. 메시지는 타입 힌트를 추가하는 역할을 합니다. `Reducer`라고 합니다.


In [5]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages


class Mydata(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages


class Mydata(TypedDict):
    messages: Annotated[list, add_messages]


mydata = Mydata(messages=["Hello, world!"])
mydata

{'messages': ['Hello, world!']}

##### 참고
1. `Annotated` 는 Python 3.9 이상에서 사용할 수 있습니다.
2. 런타임에는 `Annotated`가 무시되므로, 실제 동작에는 영향을 주지 않습니다.
3. 타입 검사도구나 IDE가 `Annotated`를 지원해야 그 효과를 볼 수 있습니다.

#### add_messages

`messages` 키는 <span style="color:#1e73be; background-color:#363a43">add_messages</span> Reducer  함수로 주석이 달려 있으며, LangGragh에게 기존 목록에 새 메시지는 append 하도록 지시합니다.  
  
주석이 없는 상태 키는 각 업데이트에 의해 덮어쓰여져 가장 최근의 값이 저장됩니다.  
  
<span style="color:#1e73be; background-color:#363a43">add_messages</span> 함수는 두개의 인자(left, right)를 받습니다. left, rigth 메시지를 병합하여 반환합니다.  
  
**주요 기능**
- 두개의 메세지 리스트를 병합합니다.
- 기본적으로 "append-only" 모드로 동작합니다.
- 동일한 ID를 가진 메세지는 덮어쓰여집니다.

**동작방식**
- `right` 메세지 중 `left` 에 동일한 ID를 가진 메시지가 있으면, `left` 메시지는 무시됩니다. `right` 메시지fh 대체됩니다.
- 그 외의 경우 `right` 메시지가 `left` 메시지에 추가됩니다.

**매개변수**
- `left` (Messages): 기존 메시지 리스트
- `right` (Messages): 병합할 메시지 리스트 또는 단일 메시지

**반환값**
- `Messages`: `right`의 메시지들이 `left`에 병합된 새로운 메시지 리스트

기본 사용방법입니다.

In [13]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
message1 = [HumanMessage(content="안녕하세요.", id="1")]
message2 = [AIMessage(content="반갑습니다.", id="2")]

merged_messages = add_messages(message1, message2)
print(merged_messages)

[HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={}, id='1'), AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}, id='2')]


동일한 ID를 가진 Message가 있을 경우 대체됩니다.

In [12]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
message1 = [HumanMessage(content="안녕하세요!", id="1")]
message2 = [AIMessage(content="반갑습니다.!", id="1")]

merged_messages = add_messages(message1, message2)
print(merged_messages)

[AIMessage(content='반갑습니다.!', additional_kwargs={}, response_metadata={}, id='1')]
